# Accessing International Debt Statistics (IDS) through World Bank Data API
## Part 1 - Query the API: indicators and locations
### *Python 3*

Date: 10/2/2019 <br>
[Data source](https://datacatalog.worldbank.org/dataset/international-debt-statistics)<br>
[Developer Information](https://datahelpdesk.worldbank.org/knowledgebase/topics/125589-developer-information)<br>
[GitHub](https://github.com/worldbank/debt-data/tree/master/api-guide)

This is first part in a series on accessing the International Debt Statistics (IDS) database through the World Bank Data API. In this part, we will query the World Bank Data API to retrieve the indicator names (including the new ones!) and location codes. 

The following code in this guide will show step-by-step how to:

1. Setup up your environment with the needed Python packages
2. Create Indicator API Queries
3. Create Location API Queries

## 1. Setup
To start, make sure you have the following packages installed on your machine. If you aren't familiar with how to install a Python package, visit each of the linked packages for instructions.

- [requests](https://realpython.com/python-requests/): Making HTTP requests in Python
- [json](https://docs.python.org/3/library/json.html): Interpreting json

Then, open up your preferred mode of writing Python. This could be in a [Jupyter Notebook](https://jupyter.org/) using [Jupyter Lab](https://blog.jupyter.org/jupyterlab-is-ready-for-users-5a6f039b8906), using a code editor (like Atom or Visual Studio) + command line, or just from the command line. Then import the required packages:

In [1]:
# Import packages
import requests
import json

## 2. Indicator API Queries


To get a data series from the World Bank Data API, you first to use an "indicator code." For example, the indicator code for long-term external debt stock is "DT.DOD.DLXF.CD." These indicator codes can be found in the [World Bank Data Catalog](https://datacatalog.worldbank.org/), but you can also use the API to explore and select indicators. In this section, we will guide you through going from a topic idea to using the World Bank Data API to select an indictor code.

More information on indicator API queries can also be found through the [Data Help Desk](https://datahelpdesk.worldbank.org/knowledgebase/articles/898599-indicator-api-queries).

#### Getting the Source ID for International Debt Statistics

To find the indicator code, we will first need to specify the API source. The World Bank Data API has numerous sources, including one specific to International Debt Statistics.

The following GET request will get us every source from the World Bank Data API. However, the request is returned in a json format that is difficult to read as is, so we will include some code to parse through it. Then we can see each source name and ID available in the World Bank Data API to find IDS.

In [2]:
# Get all sources from the World Bank API
sources = requests.get("http://api.worldbank.org/v2/sources?format=json")
sourcesJSON = sources.json()
#print(sourcesJSON) # if you want to view the JSON response as is, remove the "#" at the beginning of the print command

# Parse through the response to see the source names and ID numbers.
for i in sourcesJSON[1]:
    print(i["id"],i["name"])

1 Doing Business
5 Subnational Malnutrition Database
6 International Debt Statistics
11 Africa Development Indicators
12 Education Statistics
13 Enterprise Surveys
14 Gender Statistics
15 Global Economic Monitor
16 Health Nutrition and Population Statistics
18 IDA Results Measurement System
19 Millennium Development Goals
20 Quarterly Public Sector Debt
22 Quarterly External Debt Statistics SDDS
23 Quarterly External Debt Statistics GDDS
24 Poverty and Equity
25 Jobs
27 Global Economic Prospects
28 Global Financial Inclusion
29 The Atlas of Social Protection: Indicators of Resilience and Equity
30 Exporter Dynamics Database – Indicators at Country-Year Level
31 Country Policy and Institutional Assessment
32 Global Financial Development
33 G20 Financial Inclusion Indicators
34 Global Partnership for Education
35 Sustainable Energy for All
36 Statistical Capacity Indicators
37 LAC Equity Lab
38 Subnational Poverty
39 Health Nutrition and Population Statistics by Wealth Quintile
40 Popula

From the above result, you can see that "International Debt Statistics" is has a source ID of 6. We will add that source ID to the end of our API queries.

#### Getting the Indicator code

Now that we have source ID for International Debt Statistics we can make another request to the World Bank API to receive all of the indicator names and codes associated with that source. 

In [3]:
# Requesting the indicators for the topic External Debt
indicators = requests.get("http://api.worldbank.org/v2/indicator?format=json&source=6")
indicatorsJSON = indicators.json()

Now that we have the indicators, we can view the names and codes. The default number of results on each page is 50, but what if we want to see all the indicators on one page? Let's first print the "total" number of indicators. Then using that number we can adjust our API query.

In [4]:
# Print the total number of indicators
print("There are " + str(indicatorsJSON[0]["total"]) + " IDS indicators")

There are 497 IDS indicators


The above code shows the number of indicators in the External Debt topic. When submitting our new query to the API, we will take this into account by setting the "per_page" parameter to 500. This will allow us to view all the results with one query, instead of having to request multiple pages. Then we can parse through the result to explore the different indicator names and corresponding IDs. 

In [5]:
# Get all External Debt indicators, with a per_page parameter of 500.
indicators = requests.get("http://api.worldbank.org/v2/indicator?format=json&source=6&per_page=500")
indicatorsJSON = indicators.json()
#print(indicatorsJSON) # to view ALL of the indicators as is, remove the "#" at the beginning of the print command

# Parse through the response to see the Indicator IDs and Names
for i in indicatorsJSON[1]:
    IDSindicators = (i["id"],i["name"])
    #print(IDSindicators) # to view the indicator ids and names, remove the "#" at the beginning of the print command

The result gives us all of the External Debt indicators and their codes. You can also view the IDS indicators and codes in their hierarchical order on our [data tables](http://datatopics.worldbank.org/debt/ids/). The Analytical view shows the select indicators from the IDS publication and the Standard view shows all available indicators.
The indicator and code we want is "DT.DOD.DLXF.CD External debt stocks, long-term (DOD, current US$)." Before using this data, we need to understand its full definition. You can use an API query to get that information as well.

In [6]:
# Use the indicator code to define the "indicator" variable
indicator = "DT.DOD.DLXF.CD"

# Parse through the response to get the "sourceNote" or definition for the desired indicator
for dict_entity in indicatorsJSON[1]:
    if dict_entity["id"] == indicator:
        print(dict_entity["sourceNote"])
    else:
        pass

Long-term debt is debt that has an original or extended maturity of more than one year. It has three components: public, publicly guaranteed, and private nonguaranteed debt. Data are in current U.S. dollars.


By using the API queries above, we were able to find the IDS API source ID, sort through the World Bank Data API topics, find "External Debt," explore the corresponding indicators, and then select one indicator of interest and find its ID and definition.

## 3. Location API Queries

Now that we have the indicator code, we need to select a location. To select a location by country, region, or income level category you will need to know its 2 or 3 letter code. This section will show you how to use similar API queries to pull the location names and codes. 

For more information on location API queries visit the [Data Help Desk](https://datahelpdesk.worldbank.org/knowledgebase/articles/898590-country-api-queries).

When pulling the list of locations, I will set the per_page result to 300.

In [7]:
# Requesting the locations
locations = requests.get("http://api.worldbank.org/v2/country?format=json&source=6&per_page=300")
locationsJSON = locations.json()
#print(locationsJSON) # if you want to view ALL of the indicators as is, remove the "#" at the beginning of the print command

# Parse through the response to see the location IDs and names
for i in locationsJSON[1]:
    locationCodes = (i["id"],i["name"])
    #print(locationCodes) # to view the location ids and names, remove the "#" at the beginning of the print command

From this list of locations, we can now pick one or more locations and the corresponding location code. Such as ARG - Argentina or EAP - East Asia & Pacific. The results from this query are also saved as [location-codes csv](https://github.com/worldbank/debt-data/blob/master/api-guide/location-codes.csv)*

###### *The location-codes csv was created using the API query: http://api.worldbank.org/v2/sources/2/country/data